In [1]:
llm_config = {"model":"gpt-3.5-turbo"}

In [2]:
task = """ Write a concise but engaging blogpost about the advent of humanity and how nature took a hit because of it. Make sure the blogpost is within 100 words"""

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise blogpost (with title) on given topics. You must polist your writing based on the feedback you receive"
    "and give a refined version. Only return your final work without additional comments",
    llm_config=llm_config,
)

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [6]:
import pprint
pprint.pprint(reply)

('Title: The Impact of Humanity: A Story of Triumph and Tragedy\n'
 '\n'
 'From the advent of humanity to the present day, our progress has been '
 'remarkable. Yet, this journey has come at a cost to nature. As civilizations '
 'expanded, forests were cleared, oceans polluted, and species driven to '
 'extinction. While our advancements have brought undeniable benefits, it is '
 "crucial to recognize the toll on the planet. It's time for us to reflect on "
 'our relationship with nature and work towards a more sustainable future. '
 'Together, we can preserve the beauty and balance of our natural world for '
 'generations to come.')


## Adding Reflection

Create a critic agent to reflect on the work of the writer agent

In [7]:
critic = autogen.AssistantAgent(
    name="critic",
    is_termination_msg= lambda x:x.get("content", "").find("TERMINATE") >= 0,
    system_message="""Critic. You are responsible for reviewing the plan and the execution of the assistant. 
    Please provide constructive feedback. 
    You should ONLY use information from the assistant. 
    When the execution is done, you should say `TERMINATE`.""",
    llm_config=llm_config
)

In [8]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

critic (to Writer):

 Write a concise but engaging blogpost about the advent of humanity and how nature took a hit because of it. Make sure the blogpost is within 100 words

--------------------------------------------------------------------------------
Writer (to critic):

Title: Humanity's Footprint: The Cost of Progress

Since the advent of humanity, our impact on nature has been profound. As we have advanced technologically and culturally, nature has taken a hit. Deforestation, pollution, and climate change are just a few consequences of our actions. It's crucial to reflect on the impact we have had and work towards a more sustainable future. By being mindful of our consumption, supporting conservation efforts, and advocating for eco-friendly practices, we can strive to lessen our footprint on the planet. Let's remember that we are not just inhabitants of Earth but stewards of it.

--------------------------------------------------------------------------------
critic (to Writer):

## Nested Chat

In [9]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [11]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [12]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [13]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [14]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [15]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

critic (to Writer):

 Write a concise but engaging blogpost about the advent of humanity and how nature took a hit because of it. Make sure the blogpost is within 100 words

--------------------------------------------------------------------------------
Writer (to critic):

Title: Humanity's Footprint: The Cost of Progress

Since the advent of humanity, our impact on nature has been profound. As we have advanced technologically and culturally, nature has taken a hit. Deforestation, pollution, and climate change are just a few consequences of our actions. It's crucial to reflect on the impact we have had and work towards a more sustainable future. By being mindful of our consumption, supporting conservation efforts, and advocating for eco-friendly practices, we can strive to lessen our footprint on the planet. Let's remember that we are not just inhabitants of Earth but stewards of it.

--------------------------------------------------------------------------------

******************

c:\Users\RohitSinghChauhan\Downloads\Study\GenerativeAI\DeepLearningAI_Agentic_Design_Patterns_Autogen\.venv\lib\site-packages\autogen\agentchat\conversable_agent.py:1276: UserWarning: Cannot extract summary using reflection_with_llm: Error code: 400 - {'error': {'message': "Invalid 'messages[1].name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'messages[1].name', 'code': 'invalid_value'}}. Using an empty str as summary.
  warnings.warn(


Legal Reviewer (to critic):

As a legal reviewer, my role is to ensure that the content is legally compliant. 

- Avoid making absolute claims without supporting evidence, such as stating that "nature has taken a hit." Provide specific examples or data to back up such statements to avoid potential legal issues related to factual accuracy.
- Be cautious with language that suggests legal or scientific expertise, especially when advising on actions like supporting conservation efforts or advocating for eco-friendly practices. It's important to clarify that these are personal opinions and not professional advice.
- If mentioning specific organizations or initiatives related to conservation or eco-friendly practices, ensure that the information is up-to-date and accurately represents the activities or mission of those entities to prevent any misrepresentation.

--------------------------------------------------------------------------------

*************************************************

In [16]:
print(res.summary)

Title: Embracing Feedback: Elevating Your Narrative

With a chorus of praise for vivid characters and an engaging plot, it's clear that your storytelling resonates with readers. The descriptive language you wield so masterfully brings your world to life, a testament to your talent. However, lingering pacing hiccups in the middle act and plot inconsistencies have surfaced as areas for growth. By tightening the pacing to maintain momentum and ironing out any plot incongruities, your narrative can truly flourish. Embrace this feedback as a guiding light towards refining your work and elevating the reader's experience. Your story holds immense promise – let it shine even brighter with revisions.
